In [1]:
#!pip install --upgrade pip
#!pip install torch 
#!pip install Tensorflow

In [2]:
#!pip install --upgrade keras
#!pip uninstall transformers
#!pip install transformers

In [3]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import re
import pandas as pd
#from langdetect import detect
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, TimeoutException
from transformers import pipeline

from Sentiment_Analysis import get_sentiment
from smtp_mail_sentiment import send_email

In [4]:
def init_driver(): 
    driver_path = "C:/Users/hasee/Downloads/sentiment_analysis_proj/chromedriver.exe"
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    driver.wait = WebDriverWait(driver, 25)
    return driver

In [5]:
# Initialize the WebDriver
driver = init_driver()
LOGIN_PAGE_URL = 'https://www.instagram.com'

# Load the login page
driver.get(LOGIN_PAGE_URL)

# Find the username/email/phone number input field by XPath
username_input = driver.find_element(By.XPATH, "//input[@name='username']")

# Enter your username/email/phone number
username_input.send_keys("_momin_321")

# Find the password input field by XPath
password_input = driver.find_element(By.XPATH, "//input[@name='password']")

# Enter your password
password_input.send_keys("card@123")

# Find the login button by XPath
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")

# Click the login button
login_button.click()

# Wait for the login process to complete (you can use WebDriverWait for better synchronization)
time.sleep(5)

In [6]:
# Find the "Save info" button by its class
save_info_button = driver.find_element(By.XPATH, "//button[contains(@class, '_acan') and contains(@class, '_acap') and contains(@class, '_acas') and contains(@class, '_aj1-') and contains(@class, '_ap30') and text()='Save info']")

# Click the "Save info" button
save_info_button.click()

# Wait for some time to see the effect
time.sleep(2)

profile_main_page = 'https://www.instagram.com/thewaltonadventure/'

driver.get(profile_main_page)
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Get the initial scroll position
last_height = driver.execute_script("return document.body.scrollHeight")

# Scroll down the page repeatedly until you reach the end
n = 0
while True and n <= 2: #Scrolling 3 times
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for a short while to allow content to load
    time.sleep(3)
    
    # Calculate the new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        # If the scroll height has not changed, you've reached the end of the page
        break
    n += 1
    # Update the last scroll height
    last_height = new_height


#driver.implicitly_wait(50)
expanded_page_source = driver.page_source

soup = bs(expanded_page_source, 'html.parser')
posts = soup.find_all('a', {'class':'x1i10hfl'})

pattern = r'/p/|/reel/'

posts_links = []
for i in posts:
    link = i.get('href')
    if re.search(pattern, link):
        link = 'https://www.instagram.com'+ link
        posts_links.append(link)  

In [8]:
link_column = []
All_comments = []
for link in posts_links[0:5]: # Using only 5 posts
    post_address = link
    driver.get(post_address)
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    expanded_page_source = driver.page_source
    soup = bs(expanded_page_source, 'html.parser')
    comments = soup.find_all('div', {'class':'x9f619'})
    comments_list = []
    for i in comments:  
        try:
            f = i.find('span', {'class':'x1lliihq x1plvlek xryxfnj x1n2onr6 x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye xvs91rp xo1l8bm x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj'})
            comment = f.text
            if len(comment) > 25 and len(comment) < 500: 
                #print(comment,'\n',)
                comments_list.append(comment)
        except:
            continue
        comments_list = list(set(comments_list))
        links = [link for _ in range(len(comments_list))]
    link_column.extend(links)
    All_comments.extend(comments_list)

In [9]:
sentiments = [get_sentiment(comment) for comment in All_comments]

In [10]:
data = {'Links' : link_column,
    'Comments' : All_comments,
    'Sentiment': sentiments}
dataframe = pd.DataFrame(data)

In [15]:
dataframe.head()

,Links,Comments,Sentiment
0,https://www.instagram.com/p/CyHHGfwu_hu/,Love your photos 🤩 And thanks for inspiration ...,neutral
1,https://www.instagram.com/p/CyHHGfwu_hu/,Thx for sharing your family. Warms my heart an...,positive
2,https://www.instagram.com/p/CyHHGfwu_hu/,Love ur Zest & Enthusiasm for Family & the Int...,neutral
3,https://www.instagram.com/p/CyHHGfwu_hu/,"Glad you re-introduced yourselves, your family...",neutral
4,https://www.instagram.com/p/CyHHGfwu_hu/,It’s been 20 years since I last saw you in Las...,positive


In [16]:
neagtive_comment_df = dataframe[dataframe.Sentiment == 'negative']

In [17]:
neagtive_comment_df

,Links,Comments,Sentiment
9,https://www.instagram.com/reel/CxVwiTgrBbQ/,Instead of buying them smartphones and letting...,negative
48,https://www.instagram.com/reel/C3tFq0BO_S2/,lower center of gravity. I've been skiing sinc...,negative


In [18]:
# Example usage
sender_email = "mamoonshakeel80@gmail.com"
sender_password = "azcdhmpjxbtkwsrf"
receiver_email = "mominshakeel58896@gmail.com"

In [19]:
for i,r in neagtive_comment_df.iterrows():
    if r.Sentiment == 'negative':
        comment_text = r.Comments
        post_link = r.Links
        # Send the email
        send_email(sender_email, sender_password, receiver_email, comment_text,post_link)

Mail sent
Mail sent
